In [ ]:
import numpy as np 

In [ ]:
import numpy as np
from collections import deque 
# Simulate your episode starts and weights
batch_size, num_envs = 5, 4
episode_starts = np.random.randint(0, 2, (batch_size, num_envs))
weights = np.random.rand(batch_size, num_envs)


flat_starts = episode_starts.ravel()
flat_weights = weights.ravel()
env_boundaries = np.arange(0, len(flat_starts)+1, batch_size)
episode_boundaries = np.cumsum(episode_starts, axis=0)
cumulative_starts = np.cumsum(flat_starts)

unique_starts, unique_inverses, counts = np.unique(cumulative_starts, return_inverse=True, return_counts=True)

desired_length = 2
indices_to_sample = np.where(counts >= desired_length)[0]

In [ ]:

k_steps = 3
flat_starts = episode_starts.ravel()
flat_weights = weights.ravel()
flat_starts_indices = np.nonzero(flat_starts)[0]
steps_indices = np.arange(len(flat_starts))
diff = flat_starts_indices.reshape(-1, 1) - steps_indices
sample_mask = ~np.any((diff > 0) & (diff < k_steps), axis=0)
print(flat_starts)
print(sample_mask)

sample_mask[-k_steps+1:] = False 
indices = np.nonzero(sample_mask)[0]

In [ ]:
import numpy as np
from typing import Optional

class TestClass:
    def __init__(self):
        self.buffer_size = 5
        self.n_envs = 2
        self.n_step_bootstrapping = 2
        self.lambda_t = 1
        self.gamma_t = 1
        self.prioritized_alpha = 0.6
        self.rewards = np.array([
            [1, 2],
            [3, 4],
            [5, 6],
            [7, 8],
            [9, 10]
        ])
        self.values = np.array([
            [11, 12],
            [13, 14],
            [15, 16],
            [17, 18],
            [19, 20]
        ])
        self.episode_starts = np.array([
            [1, 1],
            [0, 0],
            [1, 0],
            [0, 0],
            [0, 1]
        ])
        self.Rn = np.zeros((self.buffer_size, self.n_envs))
        self.weights = np.zeros((self.buffer_size, self.n_envs))
    
    def compute_Rn_and_weights(
        self, 
        last_values: np.ndarray, 
        dones: np.ndarray,
        n: Optional[int] = None,
        lambda_t: Optional[float] = None,
        gamma_t: Optional[float] = None 
    ) -> None:
        n = self.n_step_bootstrapping if n is None else n 
        lambda_t = self.lambda_t if lambda_t is None else lambda_t
        gamma_t = self.gamma_t if gamma_t is None else gamma_t

        next_non_terminal = 1.0 - dones 
        next_values = last_values

        r = np.concatenate([self.rewards, np.zeros((n, self.n_envs))])
        v = np.concatenate([self.values, np.ones((n, self.n_envs)) * last_values])
        ep_starts = np.concatenate([self.episode_starts, np.ones((n, self.n_envs)) * dones])
        
        for step in reversed(range(self.buffer_size)):
            next_non_terminal = 1.0 - ep_starts[step + n]
            next_values = v[step + n]
            G = next_non_terminal * next_values
            for t in reversed(range(step, step + n)):
                non_terminal = 1.0 - ep_starts[t + 1]
                G = r[t] + gamma_t * non_terminal * (lambda_t * G + (1 - lambda_t) * v[t + 1]) 
            self.Rn[step] = G 
        self.weights = np.abs(self.values - self.Rn) ** self.prioritized_alpha

# Initialize
test_class = TestClass()

# Test the function
last_values = np.array([21, 22])
dones = np.array([0, 1])

test_class.compute_Rn_and_weights(last_values, dones)

# Output the computed Rn and weights
print('starts', test_class.episode_starts)
print('rewards:', test_class.rewards)
print('values:', test_class.values)
print("Computed Rn:", test_class.Rn)
print("Computed Weights:", test_class.weights)


In [ ]:
sample_indices = indices[:, np.newaxis] + np.arange(k_steps)

In [ ]:
anoarr = np.random.randn(batch_size, num_envs, 3)
anoarr.shape

In [ ]:
import random 
flat_starts[random.choices(sample_indices, weights=flat_weights[indices], k=3)]

In [ ]:
import numpy as np

def compute_n_step_bootstrapping(episode_starts, rewards, values, n, gamma=1, lambda_t=1.0):
    buffer_size, n_envs = rewards.shape
    
    # Initialize n-step bootstrapped returns
    n_step_returns = np.zeros_like(rewards)
    
    # Loop through each environment
    for env in range(n_envs):
        
        # Extract data for this environment
        ep_starts = episode_starts[:, env]
        r = rewards[:, env]
        v = values[:, env]
        
        # Initialize targets
        targets = np.zeros(buffer_size)
        
        # Loop through each step in reverse order
        for t in reversed(range(buffer_size - n + 1)):
            
            G = v[min(t + n, buffer_size - 1)]
            
            # Calculate n-step return
            for step in reversed(range(t, t + n)):
                if ep_starts[step]:
                    G = 0  # Reset at the start of each episode
                G = r[step] + gamma * G
            
            # Apply lambda
            targets[t] = (1 - lambda_t) * v[t] + lambda_t * G
        
        # Store in the final returns matrix
        n_step_returns[:, env] = targets
    
    return n_step_returns

# Test the function
episode_starts = np.array([
    [1, 1],
    [0, 0],
    [1, 0],
    [0, 0],
    [0, 1]
])

rewards = np.array([
    [1, 2],
    [3, 4],
    [5, 6],
    [7, 8],
    [9, 10]
])

values = np.array([
    [11, 12],
    [13, 14],
    [15, 16],
    [17, 18],
    [19, 20]
])

print(compute_n_step_bootstrapping(episode_starts, rewards, values, n=2))


In [ ]:
episode_starts

In [ ]:
%%timeit 
candidates_indices = []
candidates_weights = []
for env in range(num_envs):
    unique_boundaries, counts = np.unique(episode_boundaries[:, env], return_counts=True)
    
    # Skip 0 as it marks the start of the array
    current_index = 0
    
    for ub, count in zip(unique_boundaries[1:], counts[1:]):
        # Here you could also use your desired_length to filter out episodes that are too short
        if count >= desired_length:
            for start_idx in range(current_index, current_index + count - desired_length + 1):
                end_idx = start_idx + desired_length
                
                # Calculate average weight for this segment
                avg_weight = np.mean(weights[start_idx:end_idx, env])
                
                candidates_indices.append((start_idx, end_idx, env))
                candidates_weights.append(avg_weight)
        
        current_index += count

# Convert lists to arrays for easier manipulation
candidates_indices = np.array(candidates_indices)
candidates_weights = np.array(candidates_weights)

In [ ]:
flat_starts_indices

In [1]:
from muax.sb3.common.utils import obs_as_tensor

2023-09-14 04:09:27.577895: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import jax 
import numpy as np 

a = {'s':np.ones((3,2,4))}
jax.jit(obs_as_tensor, static_argnums=(2,))(a, is_dict_obs=type(a) == dict)

{'s': Array([[[1., 1., 1., 1.],
         [1., 1., 1., 1.]],
 
        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]],
 
        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]]], dtype=float32)}